In [27]:
!cd
!pip install -r require.txt

c:\LLM


In [28]:
import os
import glob
import json
import chromadb

from langchain_community.document_loaders import TextLoader
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.docstore.document import Document

%reload_ext dotenv
%dotenv

%run YandexLLM.ipynb
%run YandexEmbeddings.ipynb

api_key = os.getenv('YAGPT_API_KEY')
folder_id = os.getenv('YA_FOLDER_ID')
print(api_key)


print(f"API Key: {api_key}")
print(f"Folder ID: {folder_id}")

AQVNxR5vPv-B0uzNJTtvnc28vZLbP2TKu6k9fPTg
API Key: AQVNxR5vPv-B0uzNJTtvnc28vZLbP2TKu6k9fPTg
Folder ID: b1g4g4a90jln1j686ud9


In [29]:
embeding_model = YandexEmbeddings(api_key, folder_id)
llm_model = YandexLLM(api_key=api_key, folder_id=folder_id)

In [30]:
doc_files = glob.glob('C:\LLM\data_texts\*.txt')

doc_texts = []



for file_name in doc_files:
    data = open(file_name, "r").read()
    doc_texts.append(data)

# for doc in doc_texts:
#     result = embeding_model.embed_documents(doc)
#     print(result)

doc_embeddings = []
for doc in doc_texts:
    doc_embeddings.append(json.loads(embeding_model.embed_documents(doc))["embedding"])

# print(len(doc_embeddings))

In [31]:
# client.delete_collection("InternalDocs")

In [32]:
client = chromadb.Client()

collection = client.get_or_create_collection(name="InternalDocs")


collection.add(
    embeddings = doc_embeddings,
    documents = [doc_texts[0], doc_texts[1]],
    metadatas = [{"source": "register_account"},{"source": "vocation_request"}],
    ids = ["id1", "id2"]
)

Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Add of existing embedding ID: id1
Add of existing embedding ID: id2


In [33]:
query = "Как пройти собеседование на позицию Junior Data Scientist?"

query_embedding = json.loads(embeding_model.embed_query(query))["embedding"]

relevant_doc = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)

relevant_doc_lgch =  [Document(page_content=relevant_doc['documents'][0][0], metadata=relevant_doc['metadatas'][0][0])]

In [34]:
# page_content - is a key in langchain data type "Document", where to langchain expect find raw text
document_template = PromptTemplate(
    input_variables=["page_content"], #A list of the names of the variables.
    template="{page_content}" # The document template.
)

# The variable name where to put document in final prompt.
document_variable_name = "context"

# Template for prompt
template_override = """
    Представь что ты сотрудник компании MyApp.
    Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
"""

# create prompt
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template_override
)

In [35]:
llm_chain = LLMChain(llm=llm_model, prompt=prompt)

chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_template,
    document_variable_name=document_variable_name
)

In [36]:
result = chain.run(input_documents=relevant_doc_lgch, query=query)
print(result)

**Вот как можно ответить на вопрос о том, как пройти собеседование на позицию Junior Data Scientist, используя информацию из предложенного текста:**

Здравствуйте!

Я уверен(а), что могу внести значительный вклад в развитие вашей компании в качестве Junior Data Scientist благодаря своим знаниям и навыкам в области искусственного интеллекта и анализа данных. Вот как я могу это сделать:

1. **Понимание основ искусственного интеллекта:** Я хорошо разбираюсь в основных понятиях и принципах искусственного интеллекта, включая машинное обучение, нейронные сети и глубокое обучение. Я понимаю, как эти технологии могут быть применены для решения различных задач.

2. **Опыт работы с данными:** У меня есть опыт работы с большими объёмами данных и инструментами для их анализа. Я умею извлекать полезную информацию из данных и использовать её для принятия решений.

3. **Знание методов обработки естественного языка:** Я знаком(а) с методами обработки естественного языка (NLP), которые позволяют компью